In [31]:
import torch
from models import *
from utils import *

Load in the teacher models (and train the independent student model)

In [32]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg16_bn", pretrained=True)
device = 'cuda'if torch.cuda.is_available() else 'cpu'
print(eval(model.to(device),test_loader,nn.CrossEntropyLoss(),'cuda'))

Using cache found in C:\Users\PCF/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


(2.099226969822197, array(0.6879, dtype=float32))


In [33]:
teacher_model_vgg16_bn = vgg16_bn_f(True,100,model.state_dict())
def eval_f(model,test_loader,loss_func,device):
    total_loss = 0
    pred_list = None
    for i,batch in enumerate(test_loader):
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        pred,_ = model(x_batch)
        loss = loss_func(pred,y_batch.reshape(-1).long())
        pred_list = torch.cat([pred_list,torch.argmax(pred,dim=1) == y_batch]) if pred_list is not None else torch.argmax(pred,dim=1) == y_batch

        total_loss += loss.item()

    total_loss /= len(test_loader)

    return total_loss,pred_list.float().mean().cpu().numpy()
print(eval_f(teacher_model_vgg16_bn.to(device),test_loader,nn.CrossEntropyLoss(),'cuda'))

Loading pretrained weights
Mapping: block0.0.weight -> features.0.weight
Mapping: block0.1.weight -> features.1.weight
Mapping: block0.3.weight -> features.3.weight
Mapping: block0.4.weight -> features.4.weight
Mapping: block1.0.weight -> features.7.weight
Mapping: block1.1.weight -> features.8.weight
Mapping: block1.3.weight -> features.10.weight
Mapping: block1.4.weight -> features.11.weight
Mapping: block2.0.weight -> features.14.weight
Mapping: block2.1.weight -> features.15.weight
Mapping: block2.3.weight -> features.17.weight
Mapping: block2.4.weight -> features.18.weight
Mapping: block2.6.weight -> features.20.weight
Mapping: block2.7.weight -> features.21.weight
Mapping: block3.0.weight -> features.24.weight
Mapping: block3.1.weight -> features.25.weight
Mapping: block3.3.weight -> features.27.weight
Mapping: block3.4.weight -> features.28.weight
Mapping: block3.6.weight -> features.30.weight
Mapping: block3.7.weight -> features.31.weight
Mapping: block4.0.weight -> features.34

In [34]:
for epoch in range(5):
    independent_train_f(teacher_model_vgg16_bn,train_loader,nn.CrossEntropyLoss(),torch.optim.Adam(teacher_model_vgg16_bn.parameters(),0.001),device)

In [35]:
eval_f(teacher_model_vgg16_bn.to(device),test_loader,nn.CrossEntropyLoss(),'cuda')

(1.6801941364434114, array(0.5976, dtype=float32))

In [4]:
torch.save(teacher_model_vgg16_bn.state_dict(),'teacher_model_vgg16_bn.pth')

In [5]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg19_bn", pretrained=True)
device = 'cuda'if torch.cuda.is_available() else 'cpu'
print(eval(model.to(device),test_loader,nn.CrossEntropyLoss(),'cuda'))

Using cache found in C:\Users\PCF/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


(2.251624828690936, array(0.69619995, dtype=float32))


In [7]:
teacher_model_vgg19_bn = vgg19_bn_f(True,100,model.state_dict())
print(eval_f(teacher_model_vgg19_bn.to(device),test_loader,nn.CrossEntropyLoss(),'cuda'))

Loading pretrained weights
Mapping: block0.0.weight -> features.0.weight
Mapping: block0.1.weight -> features.1.weight
Mapping: block0.3.weight -> features.3.weight
Mapping: block0.4.weight -> features.4.weight
Mapping: block1.0.weight -> features.7.weight
Mapping: block1.1.weight -> features.8.weight
Mapping: block1.3.weight -> features.10.weight
Mapping: block1.4.weight -> features.11.weight
Mapping: block2.0.weight -> features.14.weight
Mapping: block2.1.weight -> features.15.weight
Mapping: block2.3.weight -> features.17.weight
Mapping: block2.4.weight -> features.18.weight
Mapping: block2.6.weight -> features.20.weight
Mapping: block2.7.weight -> features.21.weight
Mapping: block2.9.weight -> features.23.weight
Mapping: block2.10.weight -> features.24.weight
Mapping: block3.0.weight -> features.27.weight
Mapping: block3.1.weight -> features.28.weight
Mapping: block3.3.weight -> features.30.weight
Mapping: block3.4.weight -> features.31.weight
Mapping: block3.6.weight -> features.3

In [ ]:
for epoch in range(5):
    independent_train_f(teacher_model_vgg19_bn,train_loader,nn.CrossEntropyLoss(),torch.optim.Adam(teacher_model_vgg16_bn.parameters(),0.001),device)

In [ ]:
eval_f(teacher_model_vgg19_bn.to(device),test_loader,nn.CrossEntropyLoss(),'cuda')

(1.6801941364434114, array(0.5976, dtype=float32))

In [8]:
torch.save(teacher_model_vgg19_bn.state_dict(),'teacher_model_vgg19_bn.pth')

In [12]:
EPOCHS = 30

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
student_model = vgg11_bn_f(False,100)
for param in student_model.parameters():
    param.requires_grad = True

In [14]:
optimizer = torch.optim.Adam(student_model.parameters(),0.002)
loss_func = torch.nn.CrossEntropyLoss()

In [15]:
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in (range(EPOCHS)):
    tr_loss, tr_acc = independent_train_f(student_model,train_loader,loss_func,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_f(student_model,test_loader,loss_func,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))

Epoch: 1 Train Loss: 4.39 Train Accuracy: 0.03
Epoch: 2 Train Loss: 4.19 Train Accuracy: 0.04
Epoch: 3 Train Loss: 4.08 Train Accuracy: 0.05
Epoch: 4 Train Loss: 3.97 Train Accuracy: 0.06
Epoch: 5 Train Loss: 3.89 Train Accuracy: 0.07
Epoch: 6 Train Loss: 3.79 Train Accuracy: 0.08
Epoch: 7 Train Loss: 3.68 Train Accuracy: 0.09
Epoch: 8 Train Loss: 3.59 Train Accuracy: 0.11
Epoch: 9 Train Loss: 3.51 Train Accuracy: 0.12
Epoch: 10 Train Loss: 3.43 Train Accuracy: 0.13
Epoch: 11 Train Loss: 3.36 Train Accuracy: 0.14
Epoch: 12 Train Loss: 3.29 Train Accuracy: 0.16
Epoch: 13 Train Loss: 3.22 Train Accuracy: 0.17
Epoch: 14 Train Loss: 3.14 Train Accuracy: 0.18
Epoch: 15 Train Loss: 3.07 Train Accuracy: 0.19
Epoch: 16 Train Loss: 3.00 Train Accuracy: 0.21
Epoch: 17 Train Loss: 2.93 Train Accuracy: 0.22
Epoch: 18 Train Loss: 2.87 Train Accuracy: 0.24
Epoch: 19 Train Loss: 2.81 Train Accuracy: 0.25
Epoch: 20 Train Loss: 2.74 Train Accuracy: 0.26
Epoch: 21 Train Loss: 2.69 Train Accuracy: 0.28
E

In [16]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.38393998
Final Test Accuracy: 0.35619998
Final Train Loss: 2.235489661583815
Final Test Loss: 2.508881659264777


In [17]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_student_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_student_ts.csv")

In [18]:
torch.save(student_model.state_dict(),"vgg11_cifar100_student.pth")

#### 3.1.2.1: Basic Logit Matching

In [19]:
from tqdm import tqdm
student_model = vgg11_bn_f(False,100)
teacher_model_vgg16_bn = vgg16_bn_f(False,100)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_model_vgg16_bn.pth")) 
for param in student_model.parameters():
    param.requires_grad = True          
for param in teacher_model_vgg16_bn.parameters():
    param.requires_grad = False
teacher_model = teacher_model_vgg16_bn.to(device)
student_model = student_model.to(device)
optimizer = torch.optim.Adam(student_model.parameters(),lr=0.002)
loss_func = torch.nn.CrossEntropyLoss()
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in (range(EPOCHS)):
    tr_loss, tr_acc = train(student_model,teacher_model,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_f(student_model,test_loader,loss_func,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))


C:\Users\PCF\AppData\Local\Temp\ipykernel_25804\3696582385.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_mod

Epoch: 1 Train Loss: 1.67 Train Accuracy: 0.04
Epoch: 2 Train Loss: 1.62 Train Accuracy: 0.06
Epoch: 3 Train Loss: 1.59 Train Accuracy: 0.07
Epoch: 4 Train Loss: 1.56 Train Accuracy: 0.09
Epoch: 5 Train Loss: 1.53 Train Accuracy: 0.11
Epoch: 6 Train Loss: 1.50 Train Accuracy: 0.12
Epoch: 7 Train Loss: 1.47 Train Accuracy: 0.14
Epoch: 8 Train Loss: 1.45 Train Accuracy: 0.15
Epoch: 9 Train Loss: 1.43 Train Accuracy: 0.17
Epoch: 10 Train Loss: 1.41 Train Accuracy: 0.18
Epoch: 11 Train Loss: 1.40 Train Accuracy: 0.20
Epoch: 12 Train Loss: 1.38 Train Accuracy: 0.21
Epoch: 13 Train Loss: 1.36 Train Accuracy: 0.22
Epoch: 14 Train Loss: 1.34 Train Accuracy: 0.24
Epoch: 15 Train Loss: 1.32 Train Accuracy: 0.26
Epoch: 16 Train Loss: 1.31 Train Accuracy: 0.27
Epoch: 17 Train Loss: 1.29 Train Accuracy: 0.29
Epoch: 18 Train Loss: 1.28 Train Accuracy: 0.30
Epoch: 19 Train Loss: 1.27 Train Accuracy: 0.31
Epoch: 20 Train Loss: 1.26 Train Accuracy: 0.32
Epoch: 21 Train Loss: 1.25 Train Accuracy: 0.33
E

In [20]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.43612
Final Test Accuracy: 0.4001
Final Train Loss: 1.1359208230777165
Final Test Loss: 2.778662882033427


In [21]:
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_blm_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_blm_ts.csv")

In [22]:
torch.save(student_model.state_dict(),"vgg11_cifar100_blm.pth")

#### 3.1.2.2: Label Smooth Regularization

In [23]:
from tqdm import tqdm

student_model = vgg11_bn_f(False,100)
teacher_model_vgg16_bn = vgg16_bn_f(False,100)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_model_vgg16_bn.pth")) 
for param in student_model.parameters():
    param.requires_grad = True          
for param in teacher_model_vgg16_bn.parameters():
    param.requires_grad = False
teacher_model = teacher_model_vgg16_bn.to(device)
student_model = student_model.to(device)
optimizer = torch.optim.Adam(student_model.parameters(),lr=0.002)

for epoch in (range(EPOCHS)):
    tr_loss, tr_acc = train(student_model,teacher_model,train_loader,optimizer,device,types="lsr")
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_f(student_model,test_loader,loss_func,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))

C:\Users\PCF\AppData\Local\Temp\ipykernel_25804\2179157929.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_mod

Epoch: 1 Train Loss: 3.02 Train Accuracy: 0.03
Epoch: 2 Train Loss: 2.92 Train Accuracy: 0.05
Epoch: 3 Train Loss: 2.88 Train Accuracy: 0.07
Epoch: 4 Train Loss: 2.83 Train Accuracy: 0.08
Epoch: 5 Train Loss: 2.78 Train Accuracy: 0.09
Epoch: 6 Train Loss: 2.74 Train Accuracy: 0.10
Epoch: 7 Train Loss: 2.69 Train Accuracy: 0.11
Epoch: 8 Train Loss: 2.65 Train Accuracy: 0.13
Epoch: 9 Train Loss: 2.61 Train Accuracy: 0.14
Epoch: 10 Train Loss: 2.58 Train Accuracy: 0.16
Epoch: 11 Train Loss: 2.54 Train Accuracy: 0.17
Epoch: 12 Train Loss: 2.51 Train Accuracy: 0.19
Epoch: 13 Train Loss: 2.48 Train Accuracy: 0.20
Epoch: 14 Train Loss: 2.45 Train Accuracy: 0.21
Epoch: 15 Train Loss: 2.42 Train Accuracy: 0.23
Epoch: 16 Train Loss: 2.40 Train Accuracy: 0.24
Epoch: 17 Train Loss: 2.37 Train Accuracy: 0.25
Epoch: 18 Train Loss: 2.35 Train Accuracy: 0.27
Epoch: 19 Train Loss: 2.32 Train Accuracy: 0.28
Epoch: 20 Train Loss: 2.30 Train Accuracy: 0.29
Epoch: 21 Train Loss: 2.28 Train Accuracy: 0.30
E

In [24]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.3985
Final Test Accuracy: 0.3714
Final Train Loss: 2.100305021723823
Final Test Loss: 2.841451096686588


In [25]:
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_lsr_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_lsr_ts.csv")

In [26]:
torch.save(student_model.state_dict(),"vgg11_cifar100_lsr.pth")

#### 3.2.1.3: Decoupled Knowledge Distillation

In [27]:
from tqdm import tqdm

student_model = vgg11_bn_f(False,100)
teacher_model_vgg16_bn = vgg16_bn_f(False,100)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_model_vgg16_bn.pth")) 
for param in student_model.parameters():
    param.requires_grad = True          
for param in teacher_model_vgg16_bn.parameters():
    param.requires_grad = False

tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}
teacher_model = teacher_model_vgg16_bn.to(device)
student_model = student_model.to(device)
optimizer = torch.optim.Adam(student_model.parameters(),lr=0.002)
loss_func = torch.nn.CrossEntropyLoss()
for epoch in (range(EPOCHS)):
    tr_loss, tr_acc = train(student_model,teacher_model,train_loader,optimizer,device,types="dkd")
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_f(student_model,test_loader,loss_func,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))

C:\Users\PCF\AppData\Local\Temp\ipykernel_25804\3597643403.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_mod

Epoch: 1 Train Loss: 1.71 Train Accuracy: 0.03
Epoch: 2 Train Loss: 1.65 Train Accuracy: 0.05
Epoch: 3 Train Loss: 1.62 Train Accuracy: 0.07
Epoch: 4 Train Loss: 1.59 Train Accuracy: 0.08
Epoch: 5 Train Loss: 1.56 Train Accuracy: 0.10
Epoch: 6 Train Loss: 1.54 Train Accuracy: 0.11
Epoch: 7 Train Loss: 1.52 Train Accuracy: 0.13
Epoch: 8 Train Loss: 1.50 Train Accuracy: 0.14
Epoch: 9 Train Loss: 1.48 Train Accuracy: 0.15
Epoch: 10 Train Loss: 1.46 Train Accuracy: 0.17
Epoch: 11 Train Loss: 1.44 Train Accuracy: 0.18
Epoch: 12 Train Loss: 1.42 Train Accuracy: 0.19
Epoch: 13 Train Loss: 1.41 Train Accuracy: 0.21
Epoch: 14 Train Loss: 1.39 Train Accuracy: 0.22
Epoch: 15 Train Loss: 1.38 Train Accuracy: 0.23
Epoch: 16 Train Loss: 1.36 Train Accuracy: 0.25
Epoch: 17 Train Loss: 1.35 Train Accuracy: 0.26
Epoch: 18 Train Loss: 1.34 Train Accuracy: 0.27
Epoch: 19 Train Loss: 1.32 Train Accuracy: 0.28
Epoch: 20 Train Loss: 1.31 Train Accuracy: 0.29
Epoch: 21 Train Loss: 1.30 Train Accuracy: 0.30
E

In [28]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.39909998
Final Test Accuracy: 0.362
Final Train Loss: 1.2042830001057871
Final Test Loss: 2.945231762661296


In [29]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_dkd_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_dkd_ts.csv")

In [30]:
torch.save(student_model.state_dict(),"vgg11_cifar100_dkd.pth")